In [29]:
import torch # основная библиотека pytorch
import torch.nn as nn # модуль для создания нейронных сетей
import torch.optim as optim # оптимизаторы
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [30]:
df = pd.read_csv("./additional/ParisHousing.csv")

In [31]:
df

,squareMeters,numberOfRooms,hasYard,hasPool,floors,cityCode,cityPartRange,numPrevOwners,made,isNewBuilt,hasStormProtector,basement,attic,garage,hasStorageRoom,hasGuestRoom,price
0,75523,3,0,1,63,9373,3,8,2005,0,1,4313,9005,956,0,7,7559081.5
1,80771,39,1,1,98,39381,8,6,2015,1,0,3653,2436,128,1,2,8085989.5
2,55712,58,0,1,19,34457,6,8,2021,0,0,2937,8852,135,1,9,5574642.1
3,32316,47,0,0,6,27939,10,4,2012,0,1,659,7141,359,0,3,3232561.2
4,70429,19,1,1,90,38045,3,7,1990,1,0,8435,2429,292,1,4,7055052.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1726,89,0,1,5,73133,7,6,2009,0,1,9311,1698,218,0,4,176425.9
9996,44403,29,1,1,12,34606,9,4,1990,0,1,9061,1742,230,0,0,4448474.0
9997,83841,3,0,0,69,80933,10,10,2005,1,1,8304,7730,345,1,9,8390030.5
9998,59036,70,0,0,96,55856,1,3,2010,0,1,2590,6174,339,1,4,5905107.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   squareMeters       10000 non-null  int64  
 1   numberOfRooms      10000 non-null  int64  
 2   hasYard            10000 non-null  int64  
 3   hasPool            10000 non-null  int64  
 4   floors             10000 non-null  int64  
 5   cityCode           10000 non-null  int64  
 6   cityPartRange      10000 non-null  int64  
 7   numPrevOwners      10000 non-null  int64  
 8   made               10000 non-null  int64  
 9   isNewBuilt         10000 non-null  int64  
 10  hasStormProtector  10000 non-null  int64  
 11  basement           10000 non-null  int64  
 12  attic              10000 non-null  int64  
 13  garage             10000 non-null  int64  
 14  hasStorageRoom     10000 non-null  int64  
 15  hasGuestRoom       10000 non-null  int64  
 16  price              1000

In [33]:
X = df.drop('price', axis=1).values.astype(np.float32)
y = df['price'].values.astype(np.float32)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_log = np.log1p(y)

In [34]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X) # преобразовываем данные в тензоры
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.X) # кол-во строк в датасете

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] # определённая строка датасета по индексу

In [35]:
dataset = MyDataset(X_scaled, y_log)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [36]:
class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),  # Для регрессии
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)


In [37]:
model = MyModel(input_size=X_scaled.shape[1])
criterion = nn.MSELoss()  # в качестве метрики берём MSE
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [38]:
for epoch in range(100):
    for batch_X, batch_y in loader:
        preds = model(batch_X).squeeze()
        loss = criterion(preds, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.6329
Epoch 2, Loss: 0.3757
Epoch 3, Loss: 0.3772
Epoch 4, Loss: 0.7890
Epoch 5, Loss: 0.3546
Epoch 6, Loss: 0.2259
Epoch 7, Loss: 0.9844
Epoch 8, Loss: 0.4482
Epoch 9, Loss: 0.3375
Epoch 10, Loss: 0.5383
Epoch 11, Loss: 0.3560
Epoch 12, Loss: 0.2922
Epoch 13, Loss: 0.6594
Epoch 14, Loss: 0.3231
Epoch 15, Loss: 0.1917
Epoch 16, Loss: 0.2808
Epoch 17, Loss: 0.1137
Epoch 18, Loss: 0.1761
Epoch 19, Loss: 0.1337
Epoch 20, Loss: 0.1348
Epoch 21, Loss: 0.0803
Epoch 22, Loss: 0.1420
Epoch 23, Loss: 0.0779
Epoch 24, Loss: 0.1672
Epoch 25, Loss: 0.0317
Epoch 26, Loss: 0.0732
Epoch 27, Loss: 0.0966
Epoch 28, Loss: 0.0263
Epoch 29, Loss: 0.0153
Epoch 30, Loss: 0.0471
Epoch 31, Loss: 0.0596
Epoch 32, Loss: 0.0297
Epoch 33, Loss: 0.0305
Epoch 34, Loss: 0.0321
Epoch 35, Loss: 0.1306
Epoch 36, Loss: 0.0362
Epoch 37, Loss: 0.0297
Epoch 38, Loss: 0.0847
Epoch 39, Loss: 0.1178
Epoch 40, Loss: 0.0403
Epoch 41, Loss: 0.0143
Epoch 42, Loss: 0.0215
Epoch 43, Loss: 0.0371
Epoch 44, Loss: 0.02